In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
train_data = pd.read_csv('train_amex.csv')

In [3]:
train_data.head()

,session_id,DateTime,user_id,product,campaign_id,webpage_id,product_category_1,product_category_2,user_group_id,gender,age_level,user_depth,city_development_index,var_1,is_click
0,140690,2017-07-02 00:00,858557,C,359520,13787,4,NaN,10.0,Female,4.0,3.0,3.0,0,0
1,333291,2017-07-02 00:00,243253,C,105960,11085,5,NaN,8.0,Female,2.0,2.0,NaN,0,0
2,129781,2017-07-02 00:00,243253,C,359520,13787,4,NaN,8.0,Female,2.0,2.0,NaN,0,0
3,464848,2017-07-02 00:00,1097446,I,359520,13787,3,NaN,3.0,Male,3.0,3.0,2.0,1,0
4,90569,2017-07-02 00:01,663656,C,405490,60305,3,NaN,2.0,Male,2.0,3.0,2.0,1,0


In [4]:
train_data1 = pd.read_csv('historical_user_logs1_amex.csv', engine='python')

In [5]:
data_comb = pd.merge(train_data, train_data1, how='left', on=['user_id'])

In [6]:
data_comb.head()

,session_id,DateTime_x,user_id,product_x,campaign_id,webpage_id,product_category_1,product_category_2,user_group_id,gender,age_level,user_depth,city_development_index,var_1,is_click,DateTime_y,product_y,action
0,140690,2017-07-02 00:00,858557,C,359520,13787,4,NaN,10.0,Female,4.0,3.0,3.0,0,0,NaN,NaN,NaN
1,333291,2017-07-02 00:00,243253,C,105960,11085,5,NaN,8.0,Female,2.0,2.0,NaN,0,0,6/17/2017 23:20,H,view
2,129781,2017-07-02 00:00,243253,C,359520,13787,4,NaN,8.0,Female,2.0,2.0,NaN,0,0,6/17/2017 23:20,H,view
3,464848,2017-07-02 00:00,1097446,I,359520,13787,3,NaN,3.0,Male,3.0,3.0,2.0,1,0,NaN,NaN,NaN
4,90569,2017-07-02 00:01,663656,C,405490,60305,3,NaN,2.0,Male,2.0,3.0,2.0,1,0,NaN,NaN,NaN


In [7]:
test_data = pd.read_csv('test_amex.csv')

In [8]:
data_comb = data_comb.drop(['session_id','DateTime_x', 'DateTime_y', 'product_category_2','product_y'], axis=1)

In [9]:
data_comb = data_comb.drop_duplicates()

In [10]:
data_comb.head()

,user_id,product_x,campaign_id,webpage_id,product_category_1,user_group_id,gender,age_level,user_depth,city_development_index,var_1,is_click,action
0,858557,C,359520,13787,4,10.0,Female,4.0,3.0,3.0,0,0,NaN
1,243253,C,105960,11085,5,8.0,Female,2.0,2.0,NaN,0,0,view
2,243253,C,359520,13787,4,8.0,Female,2.0,2.0,NaN,0,0,view
3,1097446,I,359520,13787,3,3.0,Male,3.0,3.0,2.0,1,0,NaN
4,663656,C,405490,60305,3,2.0,Male,2.0,3.0,2.0,1,0,NaN


In [11]:
data_comb.count()

user_id                   375983
product_x                 375983
campaign_id               375983
webpage_id                375983
product_category_1        375983
user_group_id             360935
gender                    360935
age_level                 360935
user_depth                360935
city_development_index    273551
var_1                     375983
is_click                  375983
action                    166118
dtype: int64

In [12]:
data_comb.isnull().sum(axis = 0)

user_id                        0
product_x                      0
campaign_id                    0
webpage_id                     0
product_category_1             0
user_group_id              15048
gender                     15048
age_level                  15048
user_depth                 15048
city_development_index    102432
var_1                          0
is_click                       0
action                    209865
dtype: int64

In [30]:
imp = Imputer(missing_values='NaN', strategy="most_frequent")
data_comb['user_group_id'] = imp.fit_transform(data_comb['user_group_id'].reshape(-1, 1))
data_comb['gender'] = data_comb['gender'].fillna('U')
data_comb['age_level'] = imp.fit_transform(data_comb['age_level'].reshape(-1, 1))
data_comb['user_depth'] = imp.fit_transform(data_comb['user_depth'].reshape(-1, 1))
data_comb['city_development_index'] = imp.fit_transform(data_comb['city_development_index'].reshape(-1, 1))
data_comb['action'] = data_comb['action'].fillna('U')

/home/abhisek1651990/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/home/abhisek1651990/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.
/home/abhisek1651990/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """
/home/abhisek1651990/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [20]:
imp = Imputer(missing_values='NaN', strategy="most_frequent")
test_data['user_group_id'] = imp.fit_transform(test_data['user_group_id'].reshape(-1, 1))
test_data['gender'] = test_data['gender'].fillna('U')
test_data['age_level'] = imp.fit_transform(test_data['age_level'].reshape(-1, 1))
test_data['user_depth'] = imp.fit_transform(test_data['user_depth'].reshape(-1, 1))
test_data['city_development_index'] = imp.fit_transform(test_data['city_development_index'].reshape(-1, 1))

/home/abhisek1651990/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/home/abhisek1651990/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.
/home/abhisek1651990/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """
/home/abhisek1651990/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [36]:
X_train = data_comb[['product_x','campaign_id','webpage_id','product_category_1', 'user_group_id','gender','age_level','user_depth','city_development_index','var_1']]
y_train = data_comb['is_click']

In [22]:
X_test = test_data[['product','campaign_id','webpage_id','product_category_1', 'user_group_id','gender','age_level','user_depth','city_development_index','var_1']]

In [37]:
# product_x encoding and onehotencoding
label_encoder = LabelEncoder()
product_x_integer_encoded_train = label_encoder.fit_transform(X_train['product_x'])
product_x_integer_encoded_test = label_encoder.transform(X_test['product'])

onehot_encoder = OneHotEncoder(sparse=False)
product_x_integer_encoded_train = product_x_integer_encoded_train.reshape(len(product_x_integer_encoded_train), 1)
product_x_onehot_encoded_train = onehot_encoder.fit_transform(product_x_integer_encoded_train)
product_x_integer_encoded_test = product_x_integer_encoded_test.reshape(len(product_x_integer_encoded_test), 1)
product_x_onehot_encoded_test = onehot_encoder.transform(product_x_integer_encoded_test)

# # campaign_id encoding and onehotencoding
campaign_id_onehot_encoded_train = onehot_encoder.fit_transform(X_train['campaign_id'].values.reshape(-1, 1))
campaign_id_onehot_encoded_test = onehot_encoder.transform(X_test['campaign_id'].values.reshape(-1, 1))

## webpage_id encoding and onehotencoding ==> not needed
webpage_id_onehot_encoded_train = onehot_encoder.fit_transform(X_train['webpage_id'].values.reshape(-1, 1))
webpage_id_onehot_encoded_test = onehot_encoder.transform(X_test['webpage_id'].values.reshape(-1, 1))

## product_category_1	encoding and onehotencoding
product_category_1_onehot_encoded_train = onehot_encoder.fit_transform(X_train['product_category_1'].values.reshape(-1, 1))
product_category_1_onehot_encoded_test = onehot_encoder.transform(X_test['product_category_1'].values.reshape(-1, 1))

## user_group_id encoding and onehotencoding ==> not needed
user_group_id_onehot_encoded_train = onehot_encoder.fit_transform(X_train['user_group_id'].values.reshape(-1, 1))
user_group_id_onehot_encoded_test = onehot_encoder.transform(X_test['user_group_id'].values.reshape(-1, 1))

# gender encoding and onehotencoding
label_encoder = LabelEncoder()
gender_integer_encoded_train = label_encoder.fit_transform(X_train['gender'])
gender_integer_encoded_test = label_encoder.transform(X_test['gender'])

onehot_encoder = OneHotEncoder(sparse=False)
gender_integer_encoded_train = gender_integer_encoded_train.reshape(len(gender_integer_encoded_train), 1)
gender_onehot_encoded_train = onehot_encoder.fit_transform(gender_integer_encoded_train)
gender_integer_encoded_test = gender_integer_encoded_test.reshape(len(gender_integer_encoded_test), 1)
gender_onehot_encoded_test = onehot_encoder.transform(gender_integer_encoded_test)

# age_level
age_level_onehot_encoded_train = onehot_encoder.fit_transform(X_train['age_level'].values.reshape(-1, 1))
age_level_onehot_encoded_test = onehot_encoder.transform(X_test['age_level'].values.reshape(-1, 1))


# user_depth

user_depth_onehot_encoded_train = onehot_encoder.fit_transform(X_train['user_depth'].values.reshape(-1, 1))
user_depth_onehot_encoded_test = onehot_encoder.transform(X_test['user_depth'].values.reshape(-1, 1))

# city_development_index
city_development_index_onehot_encoded_train = onehot_encoder.fit_transform(X_train['city_development_index'].values.reshape(-1, 1))
city_development_index_onehot_encoded_test = onehot_encoder.transform(X_test['city_development_index'].values.reshape(-1, 1))

#var_1
var_1_onehot_encoded_train = onehot_encoder.fit_transform(X_train['var_1'].values.reshape(-1, 1))
var_1_onehot_encoded_test = onehot_encoder.transform(X_test['var_1'].values.reshape(-1, 1))




In [38]:
X_one_hotencoding = np.hstack((product_x_onehot_encoded_train, campaign_id_onehot_encoded_train,webpage_id_onehot_encoded_train,product_category_1_onehot_encoded_train,user_group_id_onehot_encoded_train,gender_integer_encoded_train,age_level_onehot_encoded_train,user_depth_onehot_encoded_train,city_development_index_onehot_encoded_train,var_1_onehot_encoded_train))

In [42]:
X_one_hotencoding_test = np.hstack((product_x_onehot_encoded_test, campaign_id_onehot_encoded_test,webpage_id_onehot_encoded_test,product_category_1_onehot_encoded_test,user_group_id_onehot_encoded_test,gender_integer_encoded_test,age_level_onehot_encoded_test,user_depth_onehot_encoded_test,city_development_index_onehot_encoded_test,var_1_onehot_encoded_test))

In [69]:
import csv 
with open('submission.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    for i in y_pred[:]:
        writer.writerow([test_data['session_id'][i],i])
        
writeFile.close()

In [83]:
# Import necessary modules

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
# Setup the hyperparameter grid
depth_range = [5,8]
# Set the parameters by cross-validation
tuned_parameters = {
'max_depth' : [5,8,15,25,30],
'n_estimators': [100, 200, 500],
'min_samples_split' :[10,15],
'min_samples_leaf' : [1, 2, 5, 10]}
# Instantiate a logistic regression classifier: logreg
rfc=RandomForestClassifier(random_state=1, class_weight='balanced')


# Instantiate the GridSearchCV object: logreg_cv
rfc_cv = GridSearchCV(rfc, tuned_parameters, cv=3, n_jobs = -1, verbose = 5,scoring='roc_auc')

# Fit it to the data
rfc_cv.fit(X_one_hotencoding, y_train)

# Print the tuned parameter and score
print("Tuned Parameters: {}".format(rfc_cv.best_params_))
print("Best score is {}".format(rfc_cv.best_score_))

Fitting 3 folds for each of 120 candidates, totalling 360 fits
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=10, n_estimators=100 
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=10, n_estimators=100 
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=10, n_estimators=100 
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=10, n_estimators=200 
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=10, n_estimators=200 
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=10, n_estimators=200 
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=10, n_estimators=500 
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=10, n_estimators=500 
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=10, n_estimators=500 
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=15, n_estimators=100 
[CV]  max_depth=5, min_samples_leaf=1, min_samples_split=10, n_estimators=100, score=0.5917367030597894, total=  43.1s
[CV] max_depth=5, min_samples_leaf=1, 

[CV] max_depth=5, min_samples_leaf=5, min_samples_split=15, n_estimators=100 
[CV]  max_depth=5, min_samples_leaf=5, min_samples_split=10, n_estimators=200, score=0.5513249903167938, total= 1.4min
[CV] max_depth=5, min_samples_leaf=5, min_samples_split=15, n_estimators=200 
[CV]  max_depth=5, min_samples_leaf=5, min_samples_split=15, n_estimators=100, score=0.5918434137386944, total=  46.9s
[CV] max_depth=5, min_samples_leaf=5, min_samples_split=15, n_estimators=200 
[CV]  max_depth=5, min_samples_leaf=5, min_samples_split=15, n_estimators=100, score=0.5730360140248947, total=  46.4s
[CV] max_depth=5, min_samples_leaf=5, min_samples_split=15, n_estimators=200 
[CV]  max_depth=5, min_samples_leaf=5, min_samples_split=15, n_estimators=100, score=0.5512419946503941, total=  45.2s
[CV] max_depth=5, min_samples_leaf=5, min_samples_split=15, n_estimators=500 
[CV]  max_depth=5, min_samples_leaf=2, min_samples_split=15, n_estimators=500, score=0.5920528904484, total= 3.8min
[CV] max_depth=5, 

[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed: 11.5min


[CV]  max_depth=5, min_samples_leaf=10, min_samples_split=10, n_estimators=100, score=0.5512308621139691, total=  46.6s
[CV] max_depth=5, min_samples_leaf=10, min_samples_split=10, n_estimators=500 
[CV]  max_depth=5, min_samples_leaf=5, min_samples_split=10, n_estimators=500, score=0.5509871291496815, total= 3.7min
[CV] max_depth=5, min_samples_leaf=10, min_samples_split=15, n_estimators=100 
[CV]  max_depth=5, min_samples_leaf=10, min_samples_split=10, n_estimators=200, score=0.5737793283805968, total= 1.5min
[CV] max_depth=5, min_samples_leaf=10, min_samples_split=15, n_estimators=100 
[CV]  max_depth=5, min_samples_leaf=10, min_samples_split=10, n_estimators=200, score=0.5919567428948151, total= 1.5min
[CV] max_depth=5, min_samples_leaf=10, min_samples_split=15, n_estimators=100 
[CV]  max_depth=5, min_samples_leaf=10, min_samples_split=10, n_estimators=200, score=0.5511439963766275, total= 1.5min
[CV] max_depth=5, min_samples_leaf=10, min_samples_split=15, n_estimators=200 
[CV]  

[CV] max_depth=8, min_samples_leaf=2, min_samples_split=15, n_estimators=200 
[CV]  max_depth=8, min_samples_leaf=2, min_samples_split=15, n_estimators=100, score=0.5796725328966236, total= 1.0min
[CV] max_depth=8, min_samples_leaf=2, min_samples_split=15, n_estimators=200 
[CV]  max_depth=8, min_samples_leaf=2, min_samples_split=15, n_estimators=100, score=0.5574582097299015, total= 1.0min
[CV] max_depth=8, min_samples_leaf=2, min_samples_split=15, n_estimators=500 
[CV]  max_depth=8, min_samples_leaf=1, min_samples_split=15, n_estimators=500, score=0.5912385967564782, total= 5.4min
[CV] max_depth=8, min_samples_leaf=2, min_samples_split=15, n_estimators=500 
[CV]  max_depth=8, min_samples_leaf=1, min_samples_split=15, n_estimators=500, score=0.5805063288346977, total= 5.3min
[CV] max_depth=8, min_samples_leaf=2, min_samples_split=15, n_estimators=500 
[CV]  max_depth=8, min_samples_leaf=1, min_samples_split=15, n_estimators=500, score=0.5583816393122883, total= 5.2min
[CV] max_depth=

...

[CV]  max_depth=25, min_samples_leaf=5, min_samples_split=10, n_estimators=500, score=0.5661202649163707, total= 8.5min
[CV] max_depth=25, min_samples_leaf=10, min_samples_split=10, n_estimators=500 
[CV]  max_depth=25, min_samples_leaf=5, min_samples_split=10, n_estimators=500, score=0.5465434686027584, total= 7.6min
[CV] max_depth=25, min_samples_leaf=10, min_samples_split=15, n_estimators=100 
[CV]  max_depth=25, min_samples_leaf=10, min_samples_split=10, n_estimators=200, score=0.5658273808770508, total= 3.2min
[CV] max_depth=25, min_samples_leaf=10, min_samples_split=15, n_estimators=100 
[CV]  max_depth=25, min_samples_leaf=10, min_samples_split=10, n_estimators=200, score=0.5703566896550416, total= 3.3min
[CV] max_depth=25, min_samples_leaf=10, min_samples_split=15, n_estimators=100 
[CV]  max_depth=25, min_samples_leaf=10, min_samples_split=15, n_estimators=100, score=0.5704038656675554, total= 1.7min
[CV] max_depth=25, min_samples_leaf=10, min_samples_split=15, n_estimators=20

[CV] max_depth=30, min_samples_leaf=2, min_samples_split=15, n_estimators=200 
[CV]  max_depth=30, min_samples_leaf=2, min_samples_split=15, n_estimators=100, score=0.5564524494182514, total= 1.6min
[CV] max_depth=30, min_samples_leaf=2, min_samples_split=15, n_estimators=200 
[CV]  max_depth=30, min_samples_leaf=2, min_samples_split=15, n_estimators=100, score=0.5416660762618792, total= 1.6min
[CV] max_depth=30, min_samples_leaf=2, min_samples_split=15, n_estimators=500 
[CV]  max_depth=30, min_samples_leaf=1, min_samples_split=15, n_estimators=500, score=0.5543096503103111, total= 8.3min
[CV] max_depth=30, min_samples_leaf=2, min_samples_split=15, n_estimators=500 
[CV]  max_depth=30, min_samples_leaf=1, min_samples_split=15, n_estimators=500, score=0.5403414025577626, total= 8.1min
[CV] max_depth=30, min_samples_leaf=2, min_samples_split=15, n_estimators=500 
[CV]  max_depth=30, min_samples_leaf=1, min_samples_split=15, n_estimators=500, score=0.5595675771157518, total= 8.5min
[CV] 

[CV]  max_depth=30, min_samples_leaf=10, min_samples_split=15, n_estimators=200, score=0.5692460181853063, total= 3.1min
[CV]  max_depth=30, min_samples_leaf=10, min_samples_split=15, n_estimators=200, score=0.5510701248160812, total= 3.0min
[CV]  max_depth=30, min_samples_leaf=10, min_samples_split=10, n_estimators=500, score=0.5693810688576322, total= 7.2min
[CV]  max_depth=30, min_samples_leaf=10, min_samples_split=10, n_estimators=500, score=0.5652536503008387, total= 7.2min
[CV]  max_depth=30, min_samples_leaf=10, min_samples_split=10, n_estimators=500, score=0.5512758370907489, total= 7.5min
[CV]  max_depth=30, min_samples_leaf=10, min_samples_split=15, n_estimators=500, score=0.5652536503008387, total= 4.9min
[CV]  max_depth=30, min_samples_leaf=10, min_samples_split=15, n_estimators=500, score=0.5512758370907489, total= 4.8min
[CV]  max_depth=30, min_samples_leaf=10, min_samples_split=15, n_estimators=500, score=0.5693810688576322, total= 6.1min


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 137.1min finished


Tuned Parameters: {'max_depth': 8, 'min_samples_leaf': 10, 'min_samples_split': 10, 'n_estimators': 500}
Best score is 0.5772746614596027


In [84]:
# Predicting the Test set results
y_pred = rfc_cv.predict(X_one_hotencoding_test)

In [89]:
import csv 
with open('submission.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    for i in y_pred[:]:
        writer.writerow([test_data['session_id'][i],i])
        
writeFile.close()